## OSRM (Open Source Routing Machine)

**Author**: SADDIK Imad
<br/>
**Date**: 02/01/2025

---

In this notebook, I will demonstrate how to modify the maximum size of points that can be fed into each service, reusing code from the match service notebook.

In [3]:
from pydantic import BaseModel


class Point(BaseModel):
    latitude: float
    longitude: float
    timestamp: int  # Unix timestamp

By default, the maximum limit is set to 100 points. However, this `GPX` file contains 1420 points, which exceeds the limit. To proceed, we can either process 100 points at a time or, preferably, increase the maximum limit.

In [4]:
import xml.etree.ElementTree as ET
from datetime import datetime


def parse_gpx_and_extract_points(file_path: str) -> list[Point]:
    # Parse the XML file into a tree structure
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Define namespace mapping for GPX 1.1 schema
    ns = {"default": "http://www.topografix.com/GPX/1/1"}

    points = []
    for trkpt in root.findall(".//default:trkpt", ns):
        lat = float(trkpt.attrib["lat"])
        lon = float(trkpt.attrib["lon"])

        # Convert ISO 8601 timestamp (removing Z and adding UTC offset) to Unix timestamp
        time_str = trkpt.find("default:time", ns).text
        timestamp = int(datetime.fromisoformat(
            time_str.replace("Z", "+00:00")).timestamp())

        points.append(Point(latitude=lat, longitude=lon, timestamp=timestamp))

    return points


file_path = "../data/gps_trace.gpx"
points = parse_gpx_and_extract_points(file_path)
len(points)

1420

In [ ]:
import requests

service = 'match'
version = 'v1'
profile = 'driving'
host = 'http://localhost:5000'

max_points_to_use = len(points)
points_portion = points[:max_points_to_use]
coordinates = ";".join(
    [f"{point.longitude},{point.latitude}" for point in points_portion])
timestamps = ";".join([str(point.timestamp)
                       for point in points_portion])

url = f"{host}/{service}/{version}/{profile}/{coordinates}"
params = {
    "steps": "true",
    "geometries": "geojson",
    "overview": "full",
    "annotations": "true",
    "timestamps": timestamps
}

response = requests.get(url, params=params)
response.json()